In [74]:
import numpy as np
import pandas as pd
Metadata = pd.read_csv('./train/Metadata.csv')

In [75]:
from tensorflow import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import glob

In [96]:
x = 150
y = 150

In [97]:
train_data = np.stack(
    [img_to_array(
        load_img(glob.glob("./train\\*\\"+i+"*.tiff")[0], target_size=(x, y, 3))) for i in Metadata['image'][:600]])

#Reshaping
train_data = train_data.reshape( len(train_data), x*y*3 )

#Normalizing
train_data = train_data / 255.0

In [98]:
train_labels = np.array(Metadata['stage'][:600])

In [99]:
test_data = np.stack(
    [img_to_array(
        load_img(glob.glob("./train\\*\\"+i+"*.tiff")[0], target_size=(x, y, 3))) for i in Metadata['image'][600:]])

#Reshaping
test_data = test_data.reshape( len(test_data), x*y*3 )

#Normalizing
test_data = test_data / 255.0

In [100]:
test_labels = np.array(Metadata['stage'][600:])

In [101]:
from keras import models
from keras import layers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(x * y * 3,)))
network.add(layers.Dense(256, activation='relu'))
network.add(layers.Dense(64, activation='relu'))
network.add(layers.Dense(3, activation='softmax'))

In [102]:
temp = []
for i in range(len(train_labels)):
    
    if train_labels[i] == "Long-duration":
        temp.append(0)
    elif train_labels[i] == "Non-diabetic":
        temp.append(1)
    else:
        temp.append(2)

train_labels = to_categorical(np.array(temp))

In [103]:
temp = []
for i in range(len(test_labels)):
    
    if test_labels[i] == "Long-duration":
        temp.append(0)
    elif test_labels[i] == "Non-diabetic":
        temp.append(1)
    else:
        temp.append(2)

test_labels = to_categorical(np.array(temp))

In [104]:
network.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [105]:
network.fit(train_data, train_labels, epochs=5, batch_size=128)

Epoch 1/5
5/5 [==============================] - 4s 779ms/step - loss: 8.2973 - accuracy: 0.3433
Epoch 2/5
5/5 [==============================] - 3s 591ms/step - loss: 3.2754 - accuracy: 0.3950
Epoch 3/5
5/5 [==============================] - 3s 600ms/step - loss: 1.9129 - accuracy: 0.3533
Epoch 4/5
5/5 [==============================] - 3s 576ms/step - loss: 1.3953 - accuracy: 0.4333
Epoch 5/5
5/5 [==============================] - 3s 625ms/step - loss: 1.5396 - accuracy: 0.4017


In [106]:
test_loss, test_acc = network.evaluate(test_data, test_labels)
print('test_acc:', test_acc)
print('test_loss:', test_loss)

8/8 [==============================] - 0s 46ms/step - loss: 1.2941 - accuracy: 0.4122
test_acc: 0.41224488615989685
test_loss: 1.2940518856048584
